In [ ]:
!pip install pyspark
!pip install pyspark[sql]

     |████████████████████████████████| 212.4MB 69kB/s 
     |████████████████████████████████| 204kB 42.0MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=a0c5852ab8b8dae20a95d0675ac3363decc9eae195b6337d6703ab7acf16ceed
  Stored in directory: /root/.cache/pip/wheels/40/1b/2c/30f43be2627857ab80062bef1527c0128f7b4070b6b2d02139
Successfully built pyspark


In [ ]:
import csv
import numpy as np
import math
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql import SQLContext
from datetime import datetime, date
import pandas as pd
from pyspark.sql import Row
import cv2
import matplotlib.pyplot as plt
from time import clock
from threading import Thread
from multiprocessing.pool import ThreadPool

In [ ]:
def loadData():

    img = cv2.imread('012345678910.png', 0)
    # plt.imshow(img)
    cells = [np.hsplit(row, 100) for row in np.vsplit(img, 50)]  # hsp:cot  vs vsp:dong => Cat tung anh nho
    x = np.array(cells)
    # print(type(x))
    # print (x.shape)
    data = x[:, :100].reshape(-1, 400).astype(np.float32)  # [:,:50] Lay 50 so dau tien
    # print(data.shape)
    data=pd.DataFrame(data)
    # print(type(data))
    k = 0
    list = []
    for i in range(1, 5001):
        list.append(k)
        if i % 500 == 0:
            k = k + 1
    # print(list)
    list = np.array(list)
    list.reshape(5000, -1)
    list = pd.DataFrame(list)
    data["400"] = list
    data=np.array(data)
    spark = SparkSession.builder.getOrCreate()
    sc = SparkContext.getOrCreate()
    distData = sc.parallelize(data)
    trainSet =distData.collect()
    testSet =distData.take(26)
    trainSet=(np.array(trainSet)).tolist()
    testSet=(np.array(testSet)).tolist()
    # print ('ve sau :',type (testSet))

    return trainSet, testSet

In [ ]:
def loadData1():

    img = cv2.imread('012345678910.png', 0)
    # plt.imshow(img)
    cells = [np.hsplit(row, 100) for row in np.vsplit(img, 50)]  # hsp:cot  vs vsp:dong => Cat tung anh nho
    x = np.array(cells)
    # print(type(x))
    # print (x.shape)
    data = x[:, :100].reshape(-1, 400).astype(np.float32)  # [:,:50] Lay 50 so dau tien
    # print(data.shape)
    data=pd.DataFrame(data)
    # print(type(data))
    k = 0
    list = []
    for i in range(1, 5001):
        list.append(k)
        if i % 500 == 0:
            k = k + 1
    # print(list)
    list = np.array(list)
    list.reshape(5000, -1)
    list = pd.DataFrame(list)
    data["400"] = list
    data=np.array(data)
    spark = SparkSession.builder.getOrCreate()
    sc = SparkContext.getOrCreate()
    distData = sc.parallelize(data)
    trainSet1 =distData.collect()
    testSet1 =distData.take(13)
    trainSet1=(np.array(trainSet1)).tolist()
    testSet1=(np.array(testSet1)).tolist()
    # print ('ve sau :',type (testSet))

    return trainSet1, testSet1

In [ ]:
def calcDistancs(pointA, pointB, numOfFeature=400):

    tmp = 0
    for i in range(numOfFeature):
        tmp += (float(pointA[i]) - float(pointB[i])) ** 2

    return math.sqrt(tmp)

In [ ]:
def kNearestNeighbor(trainSet, point, k):

    distances = []
    for item in trainSet:
        distances.append({
            "label": item[-1],
            "value": calcDistancs(item, point)
        })
    distances.sort(key=lambda x: x["value"])
    labels = [item["label"] for item in distances]
    return labels[:k]

In [ ]:
def findMostOccur(arr):

    labels = set(arr)
    ans = ""
    maxOccur = 0
    for label in labels:
        num = arr.count(label)
        if num > maxOccur:
            maxOccur = num
            ans = label


    return ans

In [ ]:
# if __name__ == "__main__":
def thucthi(x):

    trainSet, testSet = x
    numOfRightAnwser = 0
    for item in testSet:
        knn = kNearestNeighbor(trainSet, item, 5)
        answer = findMostOccur(knn)
        numOfRightAnwser += item[-1] == answer
        print("label: {} -> predicted: {}".format(item[-1], answer))
    print("Accuracy", numOfRightAnwser/len(testSet))


In [ ]:
from multiprocessing import Pool
if __name__ == "__main__":
    start1=clock()
    with Pool(15) as p:
        print(p.map(thucthi, [loadData()]))

    # with ThreadPool(5) as p:
    #     print(p.map(thucthi, [loadData()]))

    # pool = ThreadPool(5)
    # print(pool.map(thucthi, [loadData(),loadData1()]))
    end1=clock()
    duration1=end1-start1
    print ("duration can tinh la :", duration1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  This is separate from the ipykernel package so we can avoid doing imports until


label: 0.0 -> predicted: 0.0
label: 0.0 -> predicted: 0.0
label: 0.0 -> predicted: 0.0
label: 0.0 -> predicted: 0.0
label: 0.0 -> predicted: 0.0
label: 0.0 -> predicted: 0.0
label: 0.0 -> predicted: 0.0
label: 0.0 -> predicted: 0.0
label: 0.0 -> predicted: 0.0
label: 0.0 -> predicted: 0.0
label: 0.0 -> predicted: 0.0
label: 0.0 -> predicted: 0.0
label: 0.0 -> predicted: 0.0
label: 0.0 -> predicted: 0.0
label: 0.0 -> predicted: 0.0
label: 0.0 -> predicted: 0.0
label: 0.0 -> predicted: 0.0
label: 0.0 -> predicted: 0.0
label: 0.0 -> predicted: 0.0
label: 0.0 -> predicted: 0.0
label: 0.0 -> predicted: 0.0
label: 0.0 -> predicted: 0.0
label: 0.0 -> predicted: 0.0
label: 0.0 -> predicted: 0.0
label: 0.0 -> predicted: 0.0
label: 0.0 -> predicted: 0.0
Accuracy 1.0
[None]
duration can tinh la : 0.9597410000000082


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  if sys.path[0] == '':
